In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.nn import ReLU
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

In [6]:
num_classes = 10
batch_size = 64
num_epochs = 10
learning_rate = 1e-2
momentum = 0.9
weight_decay = 5e-3


In [7]:
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.49139968, 0.48215827 ,0.44653124), std=(0.24703233, 0.24348505, 0.26158768))
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

URLError: <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()

        self.layer1= nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(kernel_size=3, stride=2 )
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels=192, kernel_size=5, stride=1 , padding= 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.LocalResponseNorm(5)

        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )



        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU()
        )

        self.fc3 = nn.Sequential(
            nn.Linear(1024, num_classes),
            # nn.Softmax()
        )



    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

In [ ]:
model = AlexNet(num_classes).to(device)
model

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  )
  (layer4): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Seq

In [ ]:
cost = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate, weight_decay=weight_decay, momentum=momentum)

In [ ]:
total_step = len(trainloader)
total_step

782

In [ ]:
for epoch in range(num_epochs):
    for i , (images, labels) in enumerate(trainloader):

        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = cost(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 782 == 0:
            print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{total_step}, loss: {loss.item()}")

print('finished training')

epoch 1/10, step 782/782, loss: 1.6597864627838135
epoch 2/10, step 782/782, loss: 1.4306864738464355
epoch 3/10, step 782/782, loss: 0.9700136184692383
epoch 4/10, step 782/782, loss: 0.5285329818725586
epoch 5/10, step 782/782, loss: 1.392087459564209
epoch 6/10, step 782/782, loss: 0.37411439418792725
epoch 7/10, step 782/782, loss: 0.994047999382019
epoch 8/10, step 782/782, loss: 0.8660303950309753
epoch 9/10, step 782/782, loss: 0.16326989233493805
epoch 10/10, step 782/782, loss: 0.4928209185600281
finished training


# Test

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0

  for images, labels in trainloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted= torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  accuracy = 100 * correct/total
  print(accuracy)

77.074
